In [ ]:
# Reference: https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import random
import os
import seqeval
import numpy as np
from tqdm.auto import tqdm

In [2]:
global_seed = 42

def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
set_seed(global_seed)

In [3]:
def get_device(device_id = 0):
    if torch.cuda.is_available():    
        # Tell PyTorch to use the GPU.    
        device = torch.device(f"cuda:{device_id}")
        print('There are %d GPU(s) available.' % torch.cuda.device_count())
        print('We will use the GPU:', torch.cuda.get_device_name(device_id))
    else:
        print('No GPU available, using the CPU instead.')
        device = torch.device("cpu")
    return device


device = get_device(0)

No GPU available, using the CPU instead.


In [4]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [5]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 768
HIDDEN_DIM = 256
MINIBATCH_SIZE = 2
LEARNING_WEIGHT = 5e-2 #learning rate
WEIGHT_DECAY = 1e-4

In [6]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tmp = torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long)
#         print(tmp.is_cuda)
#         tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags]).to(device)
        tags = torch.cat([tmp.to(device), tags]).to(device)
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1).to(device)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [7]:
import pandas as pd
#data_df = pd.read_csv('BIO_tagging.csv')
data_path = "~/sr_drive/Avik/Annotations/results/finalized_dataset/"
data_df = pd.read_csv(data_path + "updated_merged_BIO_tagging_removing_verb_adjacent_pos_tags.csv")
test_data = pd.read_csv(data_path + "human_test_dataset.csv")
test_data_1 = pd.read_csv(data_path + "human_auto_test_dataset.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\somii/sr_drive/Avik/Annotations/results/finalized_dataset/updated_merged_BIO_tagging_removing_verb_adjacent_pos_tags.csv'

In [8]:
## Run this for Human Induced Dataset
import pandas as pd
data_path = "G:/my Drive/Avik/Annotations/results/finalized_dataset_human_induced/"
train_data = pd.read_csv(data_path + "train.csv")
dev_data = pd.read_csv(data_path + "dev.csv")
test_data = pd.read_csv(data_path + "test.csv")
# test_data = pd.read_csv("G:/My Drive/Avik/Launchpad/launchpad_bio_tagging_human.csv")

train_data = train_data.fillna("None")
dev_data = dev_data.fillna("None")
test_data = test_data.fillna("None")

In [9]:
word_to_ix = {}
for idx, row in tqdm(train_data.iterrows()):
    if(row['Entity'] not in word_to_ix):
        word_to_ix[row['Entity']] = len(word_to_ix)


0it [00:00, ?it/s]

In [10]:
for idx, row in tqdm(dev_data.iterrows()):
    if(row['Entity'] not in word_to_ix):
        word_to_ix[row['Entity']] = len(word_to_ix)
for idx, row in tqdm(test_data.iterrows()):
    if(row['Entity'] not in word_to_ix):
        word_to_ix[row['Entity']] = len(word_to_ix)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [11]:
ix_to_word = {idx: word for word, idx in word_to_ix.items()}

In [12]:
import operator
for k,v in sorted(word_to_ix.items(), key=operator.itemgetter(1))[:10]:
    print(k,v)

2.1.5 0
on 1
RH8.0 2
i 3
set 4
the 5
terse 6
phase 7
via 8
web 9


In [13]:
test_word = "Ubuntu"

test_word_idx = word_to_ix[test_word]
test_word_lookup = ix_to_word[test_word_idx]

print("The index of the word {} is {}.".format(test_word, test_word_idx))
print("The word with index {} is {}.".format(test_word_idx, test_word_lookup))

The index of the word Ubuntu is 2881.
The word with index 2881 is Ubuntu.


In [14]:
tag_to_ix = {}
for idx, row in train_data.iterrows():
    if(row['BIO Tag'] not in tag_to_ix):
        tag_to_ix[row['BIO Tag']] = len(tag_to_ix)
tag_to_ix[START_TAG] = len(tag_to_ix)
tag_to_ix[STOP_TAG] = len(tag_to_ix)

print(tag_to_ix)

{'O': 0, 'B-Organization': 1, 'B-Package': 2, 'B-Command': 3, 'B-Software_Component': 4, 'B-OS': 5, 'B-Error': 6, 'I-Error': 7, 'B-Extension': 8, 'I-OS': 9, 'B-Peripheral': 10, 'B-Architecture': 11, 'I-Package': 12, 'I-Software_Component': 13, 'I-Extension': 14, 'I-Peripheral': 15, 'I-Architecture': 16, 'I-Command': 17, 'I-Organization': 18, '<START>': 19, '<STOP>': 20}


In [15]:
ix_to_tag = {idx: tag for tag, idx in tag_to_ix.items()}
print(ix_to_tag)

{0: 'O', 1: 'B-Organization', 2: 'B-Package', 3: 'B-Command', 4: 'B-Software_Component', 5: 'B-OS', 6: 'B-Error', 7: 'I-Error', 8: 'B-Extension', 9: 'I-OS', 10: 'B-Peripheral', 11: 'B-Architecture', 12: 'I-Package', 13: 'I-Software_Component', 14: 'I-Extension', 15: 'I-Peripheral', 16: 'I-Architecture', 17: 'I-Command', 18: 'I-Organization', 19: '<START>', 20: '<STOP>'}


In [16]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_WEIGHT, weight_decay=WEIGHT_DECAY)

In [17]:
# train_yrs = list(range(2004, 2014))
# valid_yrs = list(range(2014, 2016))
# # test_yrs = list(range(2016, 2020))

# train_data = data_df[data_df['year'].isin(train_yrs)]
# valid_data = data_df[data_df['year'].isin(valid_yrs)]
# test_data = data_df[data_df['year'].isin(test_yrs)]
valid_data = dev_data

train_data_group = train_data.groupby(
['Bug Id'],as_index=False
)['Entity', 'POS Tag', 'BIO Tag'].agg(lambda x: list(x))

valid_data_group = valid_data.groupby(
['Bug Id'],as_index=False
)['Entity', 'POS Tag', 'BIO Tag'].agg(lambda x: list(x))

test_data_group = test_data.groupby(
['Bug Id'],as_index=False
)['Entity', 'POS Tag', 'BIO Tag'].agg(lambda x: list(x))

training_data = []
for idx, row in train_data_group.iterrows():
    training_data.append((row['Entity'], row['BIO Tag']))
    
validating_data = []
for idx, row in valid_data_group.iterrows():
    validating_data.append((row['Entity'], row['BIO Tag']))
    
testing_data = []
for idx, row in test_data_group.iterrows():
    testing_data.append((row['Entity'], row['BIO Tag']))

C:\Users\somii\AppData\Local\Temp/ipykernel_10940/122929333.py:10: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  train_data_group = train_data.groupby(
C:\Users\somii\AppData\Local\Temp/ipykernel_10940/122929333.py:14: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  valid_data_group = valid_data.groupby(
C:\Users\somii\AppData\Local\Temp/ipykernel_10940/122929333.py:18: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  test_data_group = test_data.groupby(


In [ ]:
test_data_group_1 = test_data_1.groupby(
['Bug Id'],as_index=False
)['Entity', 'POS Tag', 'BIO Tag'].agg(lambda x: list(x))

testing_data_1 = []
for idx, row in test_data_group.iterrows():
    testing_data.append((row['Entity'], row['BIO Tag']))

In [ ]:
# data_group = data_df.groupby(
# ['Bug Id'],as_index=False
# )['Entity', 'POS Tag', 'BIO Tag'].agg(lambda x: list(x))

# data_group.head()

In [ ]:
# formatted_data = []
# for idx, row in data_group.iterrows():
#     formatted_data.append((row['Entity'], row['BIO Tag']))
# print(formatted_data[0])

In [ ]:
# # train test split in terms of year
# formatted_data[0][1]

In [18]:
# from sklearn.model_selection import train_test_split
# training_data, testing_data = train_test_split(formatted_data, test_size=0.2, random_state=1234)
# training_data, validating_data = train_test_split(training_data, test_size=0.25, random_state=1234)

print("Number of sentences in the training dataset: {}".format(len(training_data)))
print("Number of sentences in the validation dataset: {}".format(len(validating_data)))
print("Number of sentences in the test dataset : {}".format(len(testing_data)))

Number of sentences in the training dataset: 64787
Number of sentences in the validation dataset: 99
Number of sentences in the test dataset : 351


In [ ]:
print(training_data[0])

In [ ]:
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_sent = precheck_sent.to(device)
    pred =  model(precheck_sent)[1]
    print('Prediction:   ', [ix_to_tag[idx] for idx in pred])
    print('Ground truth: ', training_data[0][1])
    print(training_data[0][0])

In [19]:
import numpy as np
from tqdm.auto import tqdm

In [28]:

import time
start = time.time()

losses = []
for epoch in tqdm(range(1)):  # 30 epochs used before 
    for sentence, tags in tqdm(training_data):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance of LSTM
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)
        sentence_in, targets = sentence_in.to(device), targets.to(device)


        
        # Step 3. A lot happens.  Run our forward pass to get features from BLSTM,
        # run the CRF and get the negative log likelihoods and find the best 
        # "path" through sentence with the tags using the viterbi algorithm 
        # (also part of forward pass).
        # BTW our dynamic computational graph is created with the forward pass
        # Returns the forward score - ground truth score (our loss measure)
        loss = model.neg_log_likelihood(sentence_in, targets)
        losses.append(loss.item())


        # Step 4. Compute the loss, gradients (backprop), and update the 
        # parameters by calling optimizer.step() - optimizer here is 
        # SGD for our CRF
        loss.backward()
        optimizer.step()

    if (epoch) % 10 == 0:
        print("Epoch: {} Loss: {}".format(epoch+1, np.mean(losses)))

end = time.time()

# with open('BiLSTM_runtimes.txt') as fout:
#     fout.write(f'Runtime of the program is: {end - start} seconds')

#torch.save(model.state_dict(), 'bilstm_crf.pth')

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/64787 [00:00<?, ?it/s]

Epoch: 1 Loss: 89.0332224792879


In [ ]:
# print(losses)
print(len(losses))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(len(losses)), losses)
print("Loss value are training : {}".format(losses[-1]))

In [ ]:
# with torch.no_grad():
#     postcheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
#     pred =  model(postcheck_sent)[1]
#     print('Prediction:   ', [ix_to_tag[idx] for idx in pred])
#     print('Ground truth: ', training_data[0][1])
#     print(training_data[0][0])

In [20]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from seqeval.metrics import classification_report
import numpy as np
from tqdm.auto import tqdm

def print_f1_score(y_label, y_pred):
    y_label = list(np.concatenate(y_label).flat)
    y_pred = list(np.concatenate(y_pred).flat)
    print('Micro f1 score: ', f1_score(y_label, y_pred, average='micro'))
    print('Macro f1 score: ', f1_score(y_label, y_pred, average='macro'))
    print('Weighted f1 score: ', f1_score(y_label, y_pred, average='weighted'))

def print_classwise_f1_score(y_label, y_pred, fout = None):
    y_label = list(np.concatenate(y_label).flat)
    y_pred = list(np.concatenate(y_pred).flat)
    labels=['B-Command', 'B-Error_description', 'B-Error_name', 'B-Extension',
       'B-OS', 'B-Package', 'B-URL', 'B-Ubuntu_distribution',
       'I-Error_description', 'I-Extension', 'I-URL', 'O']
    classwise_f1_score = f1_score(y_label, y_pred, average=None, labels=labels)
    if fout == None:
        print('Classwise F-1 Score')
    else:
        fout.write('Classwise F-1 Score\n')
    for label,score in zip(labels, classwise_f1_score):
        if fout == None:
            print(str(label + ' : ' + str(score)))
        else:
            fout.write(str(label + ' : ' + str(score) + '\n'))
    if fout == None:
        print()
    else:
        fout.write('\n')

def print_classification_report(y_label, y_pred):
    #y_label = list(np.concatenate(y_label).flat)
    #y_pred = list(np.concatenate(y_pred).flat)
    #sorted_labels = sorted(labels,key=lambda name: (name[2:], name[0])) # group B and I results
    print(classification_report(y_label, y_pred))#, labels=sorted_labels, digits=3))

In [ ]:
# train
with torch.no_grad():
    y_pred = []
    y_label = []
    for idx in tqdm(range(len(training_data))):
        postcheck_sent = prepare_sequence(training_data[idx][0], word_to_ix)
        postcheck_sent = postcheck_sent.to(device)
        pred =  model(postcheck_sent)[1]
        y_pred.append([ix_to_tag[idx] for idx in pred])
        y_label.append(training_data[idx][1])
    print_classification_report(y_label, y_pred)

In [ ]:
# validation
with torch.no_grad():
    y_pred = []
    y_label = []
    for idx in tqdm(range(len(validating_data))):
        postcheck_sent = prepare_sequence(validating_data[idx][0], word_to_ix)
        postcheck_sent = postcheck_sent.to(device)
        pred =  model(postcheck_sent)[1]
        y_pred.append([ix_to_tag[idx] for idx in pred])
        y_label.append(validating_data[idx][1])
    print_classification_report(y_label, y_pred)

In [35]:
# test
with torch.no_grad():
    y_pred = []
    y_label = []
    ent = []
    for idx in tqdm(range(len(testing_data))):
        postcheck_sent = prepare_sequence(testing_data[idx][0], word_to_ix)
        ent.append(testing_data[idx][0])
        postcheck_sent = postcheck_sent.to(device)
        pred =  model(postcheck_sent)[1]
        y_pred.append([ix_to_tag[idx] for idx in pred])
        y_label.append(testing_data[idx][1])
    print_classification_report(y_label, y_pred)
    

  0%|          | 0/351 [00:00<?, ?it/s]

                    precision    recall  f1-score   support

      Architecture       0.64      0.71      0.67       469
           Command       0.07      0.05      0.06       245
             Error       0.00      0.00      0.00       179
         Extension       0.36      0.36      0.36        81
                OS       0.52      0.61      0.56       448
      Organization       0.13      0.22      0.16       360
           Package       0.38      0.45      0.41       640
        Peripheral       0.02      0.04      0.03       227
Software_Component       0.00      0.00      0.00       163

         micro avg       0.32      0.36      0.34      2812
         macro avg       0.24      0.27      0.25      2812
      weighted avg       0.31      0.36      0.33      2812



In [ ]:
from seqeval.metrics import f1_score, precision_score, recall_score

In [47]:
fout_test = open('bilstm_pred.txt', 'w')

for i in (range(len(y_pred))):
    for j in (range(len(y_pred[i]))):
        fout_test.write(str(ent[i][j].encode('utf-8')) + "\t" + y_label[i][j] + "\t" + y_pred[i][j] + "\n")
    fout_test.write("\n")
fout_test.close()



In [37]:
ent[0][0]

'My'

In [ ]:
# with torch.no_grad(), open('BiLSTMCRF_metrics.txt', 'a') as fout:
#     y_pred = []
#     y_label = []
#     for idx in range(5000):
#         if(idx % 500 == 0): 
#             print(idx)
#         postcheck_sent = prepare_sequence(testing_data[idx][0], word_to_ix)
#         postcheck_sent = postcheck_sent.to(device)
#         pred =  model(postcheck_sent)[1]
#         y_pred.append([ix_to_tag[idx] for idx in pred])
#         y_label.append(testing_data[idx][1])
#     print_classwise_f1_score(y_label, y_pred, fout)
#     print_f1_score(y_label, y_pred)

#### Evaluation on the trained model

In [ ]:
torch.save(model, 'G:/My Drive/Avik/NER_Baselines/BiLSTM_CRF_human_induced_model.pth')

In [21]:
model = torch.load('G:/My Drive/Avik/NER_Baselines/BiLSTM_CRF_human_induced_model.pth')
model = model.to(device)

In [23]:
## This code segment helps in finding out the accuracy of the model on the human data
y_test = y_label
classwise_acc_dict = {}
classwise_count_dict = {}
for i in tqdm(range(len(y_test))):
    for j in range(len(y_test[i])):
        if y_test[i][j] != 'O':
            ent = y_test[i][j][2:]
            
            if ent not in classwise_count_dict:
                classwise_count_dict[ent] = 0
            classwise_count_dict[ent] = classwise_count_dict[ent] + 1
            
            if ent not in classwise_acc_dict:
                classwise_acc_dict[ent] = 0
            if y_pred[i][j][2:] == ent:
                classwise_acc_dict[ent] = classwise_acc_dict[ent] + 1
sum_correct = 0
for key in classwise_acc_dict.keys():
    sum_correct += classwise_acc_dict[key]
    classwise_acc_dict[key] = classwise_acc_dict[key]/classwise_count_dict[key]
print("The classwise accuracy : ")
print(classwise_acc_dict)
overall_accuracy = sum_correct/sum(classwise_count_dict.values())
print(f"The overall accuracy : {overall_accuracy}")

  0%|          | 0/351 [00:00<?, ?it/s]

The classwise accuracy : 
{'Command': 0.06, 'OS': 0.6538461538461539, 'Package': 0.450229709035222, 'Architecture': 0.6983122362869199, 'Organization': 0.34782608695652173, 'Peripheral': 0.12186978297161936, 'Error': 0.016881827209533268, 'Extension': 0.35802469135802467, 'Software_Component': 0.004807692307692308}
The overall accuracy : 0.31449333871618895
